In [ ]:
pip install umap

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import cv2
import matplotlib.pyplot as plt
import os
import seaborn as sns
import umap
from PIL import Image
from scipy import misc
from os import listdir
from os.path import isfile, join
import numpy as np
from scipy import misc
from random import shuffle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical

In [ ]:
os.listdir('/content/drive/MyDrive/salim ridho/data set/cell_images/cell_images')

In [ ]:
yes=os.listdir('/content/drive/MyDrive/salim ridho/data set/cell_images/cell_images/Parasitized')
no=os.listdir('/content/drive/MyDrive/salim ridho/data set/cell_images/cell_images/Uninfected')

In [ ]:
data=np.concatenate([yes,no])
len(data)==len(yes)+len(no)

In [ ]:
target_x=np.full(len(yes),1)
target_y=np.full(len(no),0)
data_target=np.concatenate([target_x,target_y])
len(data_target)==len(target_x)+len(target_y)

In [ ]:
len(data_target)==len(data)

In [ ]:
data_target.shape

In [ ]:
data

In [ ]:
yes_values=os.listdir('/content/drive/MyDrive/salim ridho/data set/cell_images/cell_images/Parasitized')
no_values=os.listdir('/content/drive/MyDrive/salim ridho/data set/cell_images/cell_images/Uninfected')

In [ ]:
X_data =[]

for file in yes_values:
    #face = misc.imread('../input/brain_tumor_dataset/yes/'+file)
    img = cv2.imread('/content/drive/MyDrive/cita skripsi/coba besar/Training/notumor/'+file)
    face = cv2.resize(img,(32,32),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
    (b, g, r)=cv2.split(face) 
    img=cv2.merge([r,g,b])
    X_data.append(img)

In [ ]:
for file in yes_values:
    #face = misc.imread('../input/brain_tumor_dataset/yes/'+file)
    img = cv2.imread('/content/drive/MyDrive/cita skripsi/coba besar/Training/notumor'+file)
    face = cv2.resize(img, (32, 32) )
    (b, g, r)=cv2.split(face) 
    img=cv2.merge([r,g,b])
    X_data.append(img)

In [ ]:
len(X)==len(data)==len(data_target)

In [ ]:
len(X)

In [ ]:
X = np.squeeze(X_data)

In [ ]:
X.shape

In [ ]:
# normalize data
X = X.astype('float32')
X /= 255

In [ ]:
data_target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,X_t,y_train,y_t = train_test_split(X,data_target, test_size=0.2,random_state=0)

In [ ]:
x_valid,x_test,y_valid,y_test = train_test_split(X_t,y_t, test_size=0.5,random_state=0)

In [ ]:
x_test.shape

In [ ]:
x_valid.shape

In [ ]:
model = tf.keras.Sequential()

# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=9, padding='same', activation='relu', input_shape=(32,32,3))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.45))

model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=9,padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(filters=36, kernel_size=9, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))


model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['acc'])

# Take a look at the model summary
model.summary()

In [ ]:
results = model.fit(x_train, y_train,
         batch_size=128,
         epochs=20,
         validation_data=(x_valid, y_valid),)

In [ ]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])

In [ ]:
acc=model.evaluate( x_test,y_test, verbose=0, return_dict=False)[1]*100

In [ ]:
acc

In [ ]:
labels =["meningioma",  # index 0
        "notumor",      # index 1
        ]

In [ ]:
y_hat = model.predict(x_test)

# Plot a random sample of 10 test images, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = np.argmax(y_hat[index])
    true_index = np.argmax(y_test[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(labels[predict_index], 
                                  labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(1)  
# summarize history for accuracy  
plt.subplot(211)  
plt.plot(results.history['acc'])  
plt.plot(results.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   
 # summarize history for loss  
   
plt.subplot(212)  
plt.plot(results.history['loss'])  
plt.plot(results.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()

In [ ]:
pred_probabilities = model.predict(x_test)

In [ ]:
pred_probabilities

In [ ]:
x_test,y_test

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
# Numpy can treat this as True/False for us
predictions

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(8,5))
sns.heatmap(confusion_matrix(y_test, predictions), annot=True, fmt=".0f", ax=ax)
plt.xlabel("y_head")
plt.ylabel("y_true")
plt.show()

In [ ]:
model.save('modelcita.h5')